In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

## 予測をする

In [ ]:
# 予測を行う。一つの関数・変数（コア数・各種ベンチマーク由来の初期化変数）

benchmarkNames = ["cg"]
benchmarkName  = "cg"
classes = ["S", "W", "A", "B", "C", "D", "E", "F"]
targetClass = "F"
processes = [1, 2, 4, 8, 16, 32, 64, 128, 256]
targetProcess = 256

# データを取得
rawDF = returnCollectedExistingData(
    benchmarkNames=benchmarkNames,
    classes=classes,
    processes=processes,
    csvDirPath="./csv_files/",
)
rawDF = addInitDataToRawDF(rawDF)

# モデルを用いた処理を実施
functionNames = sorted(list(set(rawDF["functionName"])))
for functionName in functionNames:
    print(f"----------functionName={functionName}----------")

    ##関数ごとにrawDFを抽出
    DFExtractedByFunction = rawDF[rawDF["functionName"]==functionName].copy()
    ## 説明変数用に問題サイズ列を数値化した列を追加する
    strListProblemSize = DFExtractedByFunction["benchmarkClass"].tolist()
    intListProblemSize = convertBenchmarkClasses_problemSizeInNPB(inputList=strListProblemSize)
    DFExtractedByFunction["intBenchmarkClass"] = intListProblemSize
    ## 3モデルでMAPEを出力
    ### 目的変数のリストを作成
    resVar = ["functionCallNum"]
    ### 重回帰分析
    #### 列名のリストをDFから取得
    expVar = DFExtractedByFunction.columns.tolist()
    #### 除外する列名を除く
    for removeElement in ["functionName", "functionCallNum", "benchmarkName", "benchmarkClass"]:
        expVar.remove(removeElement)
    #### 3モデル（線形・反比例・対数）を同時に作成
    threeModelsByMulti = Models(
        inputDF=DFExtractedByFunction,
        expVarColNames=expVar,
        resVarColNames=resVar,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    )
    threeModelsByMulti.setUpDataBeforeCalcLr()
    threeModelsByMulti.calcLr()
    threeModelsByMulti.calcMAPE()
    dictCalcedMAPE = threeModelsByMulti.returnCalculatedMAPE()
    # dictCalcedMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
    print(f"multi  = {dictCalcedMAPE}")
    ### TODO：単回帰分析
    expVar = ["intBenchmarkClass"]
    threeModelsBySingle = Models(
        inputDF=DFExtractedByFunction,
        expVarColNames=expVar,
        resVarColNames=resVar,
        targetDF=None,
        modelNames=["modelLin", "modelIp", "modelLog"],
    )
    threeModelsBySingle.setUpDataBeforeCalcLr()
    threeModelsBySingle.calcLr()
    threeModelsBySingle.calcMAPE()
    dictCalcedMAPE = threeModelsBySingle.returnCalculatedMAPE()
    # dictCalcedMAPEの数値を小数第一位までにする
    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = int(dictCalcedMAPE[key] * 10) / 10
    print(f"single = {dictCalcedMAPE}")
# TODO:予測の統計的結果をグラフにプロットする


# データを表示
print(DFExtractedByFunction)